## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.45,-133.04,46.40,65,75,12.75,broken clouds
1,1,Busselton,AU,-33.65,115.33,45.00,92,0,1.83,clear sky
2,2,Rikitea,PF,-23.12,-134.97,68.54,61,0,3.96,clear sky
3,3,Colares,PT,38.80,-9.45,71.01,94,3,18.77,clear sky
4,4,Port Alfred,ZA,-33.59,26.89,58.14,72,0,13.15,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Colares,PT,38.80,-9.45,71.01,94,3,18.77,clear sky
9,9,Coahuayana,MX,18.73,-103.68,71.60,83,90,6.42,drizzle
12,12,Tura,IN,25.52,90.22,73.98,90,28,5.84,scattered clouds
15,15,Avarua,CK,-21.21,-159.78,77.00,57,45,11.41,scattered clouds
17,17,Morondava,MG,-20.28,44.28,72.25,76,0,7.67,clear sky
19,19,Souillac,MU,-20.52,57.52,71.60,73,40,2.24,shower rain
22,22,Hithadhoo,MV,-0.60,73.08,81.75,66,100,0.78,overcast clouds
29,29,Hashtgerd,IR,35.96,50.69,75.20,25,40,4.70,scattered clouds
32,32,Bintulu,MY,3.17,113.03,75.20,100,20,6.08,few clouds
34,34,Lorengau,PG,-2.02,147.27,84.85,69,2,6.78,light rain


In [16]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isnull().values.any()
#preferred_cities_df.isnull().count()
#len(preferred_cities_df)

False

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Colares,PT,38.80,-9.45,71.01,94,3,18.77,clear sky
9,9,Coahuayana,MX,18.73,-103.68,71.60,83,90,6.42,drizzle
12,12,Tura,IN,25.52,90.22,73.98,90,28,5.84,scattered clouds
15,15,Avarua,CK,-21.21,-159.78,77.00,57,45,11.41,scattered clouds
17,17,Morondava,MG,-20.28,44.28,72.25,76,0,7.67,clear sky
...,...,...,...,...,...,...,...,...,...,...
682,682,Terekli-Mekteb,RU,44.17,45.87,72.79,57,0,3.51,clear sky
683,683,Shirdi,IN,19.77,74.48,73.83,85,94,4.16,overcast clouds
684,684,Kahului,US,20.89,-156.47,89.60,52,40,28.86,scattered clouds
687,687,Altamont,US,42.21,-121.74,84.99,20,1,6.93,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Colares,PT,71.01,clear sky,38.80,-9.45,
9,Coahuayana,MX,71.60,drizzle,18.73,-103.68,
12,Tura,IN,73.98,scattered clouds,25.52,90.22,
15,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,
17,Morondava,MG,72.25,clear sky,-20.28,44.28,
19,Souillac,MU,71.60,shower rain,-20.52,57.52,
22,Hithadhoo,MV,81.75,overcast clouds,-0.60,73.08,
29,Hashtgerd,IR,75.20,scattered clouds,35.96,50.69,
32,Bintulu,MY,75.20,few clouds,3.17,113.03,
34,Lorengau,PG,84.85,light rain,-2.02,147.27,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print (f"nearest_hotel{hotels['results'][0]['name']}")
    except (IndexError):
        print("Hotel not found... skipping.")


nearest_hotelArribas Sintra Hotel
nearest_hotelHotel Los Arcos
nearest_hotelHotel RIKMAN Continental
nearest_hotelParadise Inn
nearest_hotelLa Case Bambou
nearest_hotelBils' Villa
nearest_hotelScoop Guest House
nearest_hotelParse System
nearest_hotel900 Boutique INN
nearest_hotelLorengau Harbourside Hotel
nearest_hotelMission Of The Sun Hotel
Hotel not found... skipping.
nearest_hotelAbaco Beach Resort
nearest_hotelHilo Hawaiian Hotel
nearest_hotelINN NEW HORIZON
Hotel not found... skipping.
nearest_hotelInubosaki Kanko Hotel
nearest_hotelGreenbrier Motel & Restaurant
nearest_hotelKeleti Beach Resort
nearest_hotelIsles Sunset Lodge
nearest_hotelFakhamat Al Orjoana Apartment
nearest_hotelJinjiang Inn Zhoushan Putuo Shenjiamen
nearest_hotelALETHEIA Appartement Sinnamary
nearest_hotelCititel Penang
nearest_hotelPension Oaoa
nearest_hotelWarm Mineral Springs Motel
nearest_hotelVS Villa
nearest_hotelHotel Serras de Paraúna
nearest_hotelHampton Inn & Suites Jamestown
nearest_hotelNuba Nile
n

KeyboardInterrupt: 

In [22]:
hotel_df.head(12)
#hotel_df.count()
#hotel_df["Hotel Name"].isnull().values.any()
#hotel_df["Hotel Name"].notnull().count()
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna("Hotel Name")

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Colares,PT,71.01,clear sky,38.80,-9.45,Arribas Sintra Hotel
9,Coahuayana,MX,71.60,drizzle,18.73,-103.68,Hotel Los Arcos
12,Tura,IN,73.98,scattered clouds,25.52,90.22,Hotel RIKMAN Continental
15,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,Paradise Inn
17,Morondava,MG,72.25,clear sky,-20.28,44.28,La Case Bambou
19,Souillac,MU,71.60,shower rain,-20.52,57.52,Bils' Villa
22,Hithadhoo,MV,81.75,overcast clouds,-0.60,73.08,Scoop Guest House
29,Hashtgerd,IR,75.20,scattered clouds,35.96,50.69,Parse System
32,Bintulu,MY,75.20,few clouds,3.17,113.03,900 Boutique INN
34,Lorengau,PG,84.85,light rain,-2.02,147.27,Lorengau Harbourside Hotel


In [27]:
dupRow = hotel_df.loc[hotel_df['Hotel Name'] == '']
len(dupRow)

23

In [28]:
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Colares,PT,71.01,clear sky,38.80,-9.45,Arribas Sintra Hotel
9,Coahuayana,MX,71.60,drizzle,18.73,-103.68,Hotel Los Arcos
12,Tura,IN,73.98,scattered clouds,25.52,90.22,Hotel RIKMAN Continental
15,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,Paradise Inn
17,Morondava,MG,72.25,clear sky,-20.28,44.28,La Case Bambou
...,...,...,...,...,...,...,...
682,Terekli-Mekteb,RU,72.79,clear sky,44.17,45.87,"Gostevoy Dvor ""U Gul'zary"""
683,Shirdi,IN,73.83,overcast clouds,19.77,74.48,Shradha Saburi Palace
684,Kahului,US,89.60,scattered clouds,20.89,-156.47,Maui Seaside Hotel
687,Altamont,US,84.99,clear sky,42.21,-121.74,Microtel Inn & Suites by Wyndham Klamath Falls


In [31]:
# 8a. Create the output File (CSV)
#output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_Vacation.csv", index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))